In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" xformers datasets accelerate

In [2]:
import unsloth
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-2-7b-chat-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
custom_text_data = [
    "Hawaii wildfires have caused significant damage to the island community. The fires spread rapidly due to dry conditions and strong winds.",
    "The Maui fires in 2023 were among the deadliest in Hawaiian history. Lahaina town was severely affected by the wildfire.",
    "Climate change and drought conditions contributed to the severity of the Hawaii fires. Emergency responders worked tirelessly to evacuate residents.",
    "The economic impact of Hawaii fires was substantial, affecting tourism and local businesses. Recovery efforts are ongoing.",
    "Hawaii's unique ecosystem makes it vulnerable to wildfires. Prevention and preparedness are crucial for future fire seasons.",
    "The community response to Hawaii fires showed resilience and solidarity. Volunteers and organizations provided essential support.",
    "Wildfire prevention in Hawaii requires careful land management and early warning systems. Education about fire safety is important.",
]

In [14]:
train_dataset = Dataset.from_dict({"text": custom_text_data})

In [15]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [16]:
tokenized_train_dataset = []
for phrase in train_dataset:
    tokenized_train_dataset.append(tokenizer(phrase['text']))

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.9.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [18]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=TrainingArguments(
        output_dir="unsloth_output",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=1e-4,
        max_steps=20,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./logs",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10,
        report_to="none",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [19]:
model.config.use_cache = False

In [20]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7 | Num Epochs = 20 | Total steps = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,988,480 of 6,758,404,096 (0.30% trained)


Step,Training Loss
10,1.712500
20,0.512800


TrainOutput(global_step=20, training_loss=1.1126646995544434, metrics={'train_runtime': 106.0814, 'train_samples_per_second': 1.508, 'train_steps_per_second': 0.189, 'total_flos': 174245815468032.0, 'train_loss': 1.1126646995544434, 'epoch': 20.0})

In [21]:
model.save_pretrained("unsloth_custom_model")
tokenizer.save_pretrained("unsloth_custom_model")

('unsloth_custom_model/tokenizer_config.json',
 'unsloth_custom_model/special_tokens_map.json',
 'unsloth_custom_model/chat_template.jinja',
 'unsloth_custom_model/tokenizer.model',
 'unsloth_custom_model/added_tokens.json',
 'unsloth_custom_model/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [23]:
def test_model(question):
    eval_prompt = f"Question: {question} Just answer this question accurately."
    inputs = tokenizer(eval_prompt, return_tensors="pt").to('cuda')

    model.eval()
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=1024)[0]
        return tokenizer.decode(output, skip_special_tokens=True)

In [24]:
result = test_model("Tell Me About the Hawaii Fires?")
print(f"Result: {result}")

Result: Question: Tell Me About the Hawaii Fires? Just answer this question accurately.

Hawaii fires have caused significant damage to the island community.

The fires spread rapidly due to dry conditions and strong winds.

The fire season in Hawaii is particularly dangerous due to the island's unique landscape.

The fires were exacerbated by the presence of dry brush and vegetation.

The Hawaii fires resulted in the evacuation of hundreds of residents.

The firefighters worked tirelessly to contain the blaze.

The Hawaii fires are a reminder of the importance of fire safety in the island community.

The fires were eventually brought under control through the efforts of firefighters and residents.

The Hawaii fires caused significant damage to the island community.

The fires highlighted the need for better fire safety measures in the island community.

The Hawaii fires were a tragic reminder of the dangers of wildfires in the island community.


In [25]:
torch.cuda.empty_cache()